# VOQC: A Verifed Optimizer for Quantum Circuits <img src="tutorial-files/umd.jpeg" width="100" align="right">

Welcome to the tutorial for VOQC! 

VOQC is a **verified optimizer for quantum circuits**, implemented and *formally verified* in the [Coq proof assistant](https://coq.inria.fr/). All VOQC optimizations are *guaranteed* to preserve the semantics of the original circuit, meaning that any optimized circuit produced by VOQC has the same behavior as the input circuit. VOQC was presented as a [distinguished paper at POPL 2021](https://arxiv.org/abs/1912.02250). All of our code is available freely online.
* Our Coq definitions and proofs are available at \url{https://github.com/inQWIRE/SQIR}.
* Our OCaml library is available at https://github.com/inQWIRE/mlvoqc and can be installed with `opam install voqc`. 
* Documentation on the OCaml library interface is available at https://inqwire.github.io/mlvoqc/voqc/Voqc/index.html.
* Our Python bindings and tutorials are available at https://github.com/inQWIRE/pyvoqc.

To run VOQC in Python, we (1) extract the verified Coq code to OCaml, (2) compile the extracted OCaml code to a library, (3) wrap the OCaml library in C, and (4) use Python's ctypes to make calls into the C library. This tutorial introduces the optimizations available in VOQC through our Python interface ("pyvoqc"). For convenience, we have written code that makes VOQC look like an optimization pass in [IBM's Qiskit framework](https://qiskit.org/documentation/getting_started.html), allowing us to take advantage of Qiskit's utilities for quantum programming. In this tutorial, we'll use Qiskit for building and printing circuits.

## Outline

* End-to-end Example
* Optimizations
  * Not Propagation
  * Single-qubit Gate Cancellation
  * Two-qubit Gate Cancellation
  * Rotation Merging
  * Hadamard Reduction 

We begin with an example of reading in a circuit, optimizing it using VOQC, and writing it back to a file.
We then describe different optimizations available in VOQC, along with examples of how to run them in Qiskit.  

## Preliminaries

Follow the installaton instructions in the pyvoqc directory. (If you already have opam installed, this should be as simple as running `opam install voqc` and `./install.sh`.)

To begin the tutorial, import modules below from Qiskit and VOQC. 

In [3]:
# Click on this cell and press Ctrl+Enter or run it with the "Run" button
from qiskit import QuantumCircuit
from pyvoqc.voqc import VOQC
from pyvoqc.qiskit.voqc_pass import QiskitVOQC
from qiskit.transpiler import PassManager
from qiskit.qasm import pi

## End-to-end Example

Our Python interface allows us to pass a Qiskit circuit object through VOQC and receive an optimized Qiskit circuit.

VOQC can be called just like Qiskit's built-in transpiler passes (e.g. "Commutative Cancellation" or "CX Cancellation"). Simply append `QiskitVOQC([opt list])` to a pre-defined `PassManager` where `opt list` is an optional argument specifying one or more of the optimizations in VOQC (see *Optimizations* below). `QiskitVOQC()` with no arguments will run all optimizations available in VOQC.

In the example here, we first read in a small quantum circuit from the qasm file "tof_3_example.qasm". This circuit consists of three 3-qubit CCX (Toffoli) gates that together perform a 4-qubit Toffoli gate. We first *decompose* these 3-qubit gates into 1-qubit and CX gates using "decompose_to_cnot" (technically this is not required--it will be done by subsequent optimizations). We then apply our most general optimization functions, "optimize_nam" and "optimize_ibm".

In [5]:
# Read circuit from file
circ = QuantumCircuit.from_qasm_file("tutorial-files/tof_3_example.qasm")
print("Before Optimization:")
print(circ)

# Create a Qiskit PassManager
pm = PassManager()

# Decompose the CCX gates
pm.append(QiskitVOQC(["decompose_to_cnot"]))
circ2 = pm.run(circ)
print("\nAfter 'decompose_to_cnot':\n")
print('gates = ', circ2.count_ops())
print('circuit depth = ', circ2.depth())

# Run optimize_nam and optimize_ibm
pm.append(QiskitVOQC(["optimize_nam", "optimize_ibm"]))
circ3 = pm.run(circ2)

# Print info about optimized circuit 
print("\nAfter 'optimize_nam' and 'optimize_ibm':\n")
print('gates = ', circ3.count_ops())
print('circuit depth = ', circ3.depth())

Before Optimization:
                    
q_0: ──■─────────■──
       │         │  
q_1: ──■─────────■──
       │         │  
q_2: ──┼────■────┼──
       │  ┌─┴─┐  │  
q_3: ──┼──┤ X ├──┼──
     ┌─┴─┐└─┬─┘┌─┴─┐
q_4: ┤ X ├──■──┤ X ├
     └───┘     └───┘

After 'decompose_to_cnot':

gates =  OrderedDict([('cx', 18), ('t', 12), ('tdg', 9), ('h', 6)])
circuit depth =  33

After 'optimize_nam' and 'optimize_ibm':

gates =  OrderedDict([('cx', 16), ('u1', 14), ('u2', 6)])
circuit depth =  28


## Unitary Optimizations

Appending `QiskitVOQC()` to the pass manager runs the VOQC's main optimization function, which runs all its unitary optimizations in a predefined order. You can also run optimizations individually, or in a custom order, by passing arguments to the `QiskitVOQC` class (e.g. `QiskitVOQC(["not_propagation"])` to run "not propagation"). VOQC provides several unitary optimizations, including: *not propagation*, *single-qubit gate cancellation*, *two-qubit gate cancellation*, *rotation merging*, and *hadamard reduction*.

We provide a brief example of each optimization below. For more details see Section 4 of [our paper](https://arxiv.org/abs/1912.02250) or an earlier paper by [Nam et al. (2018)](https://www.nature.com/articles/s41534-018-0072-4), which inspired many of VOQC's optimizations.

The optimization described here all use our "RzQ" gate set, which uses the non-standard rzq gate. To avoid compatibility issues, we recommend running "replace_rzq" after any of these optimizations.

### Not Propagation

*Not propagation* commutes X (logical NOT) gates rightward through the circuit, cancelling them when possible. In the example below, the leftmost X gate propagates through the CNOT gate to become two X gates. The upper X gate then propagates through the H gate to become a Z, and the two lower X gates cancel.

In [7]:
# Build circuit with 2 qubits and 4 gates
circ = QuantumCircuit(2)
circ.x(0)
circ.cx(0, 1)
circ.h(0)
circ.x(1)
print("Before Optimization:")
print(circ)

# Append "not_propagation" optimization to the Pass Manager
pm = PassManager()
pm.append(QiskitVOQC(["not_propagation", "replace_rzq"]))
new_circ = pm.run(circ)

# Print optimized circuit
print("\nAfter Optimization:")
print(new_circ)

Before Optimization:
     ┌───┐     ┌───┐
q_0: ┤ X ├──■──┤ H ├
     └───┘┌─┴─┐├───┤
q_1: ─────┤ X ├┤ X ├
          └───┘└───┘

After Optimization:
          ┌───┐┌───┐
q_0: ──■──┤ H ├┤ Z ├
     ┌─┴─┐└───┘└───┘
q_1: ┤ X ├──────────
     └───┘          


### Single-qubit Gate Cancellation

*Single-qubit gate cancellation* has the same "propagate-cancel" structure as not propagation, except that gates revert back to their original positions if they fail to cancel. In the example below, the upper leftmost T gate commutes through the control of the CNOT, combining with the upper rightmost T gate to become an S gate. The lower T gate commutes through the H; CNOT; H subcircuit to cancel with the Tdg gate.

In [9]:
# Build circuit with 2 qubits and 7 gates
circ = QuantumCircuit(2)
circ.t(0)
circ.t(1)
circ.h(1)
circ.cx(0, 1)
circ.h(1)
circ.t(0)
circ.tdg(1)
print("Before Optimization:")
print(circ)

# Append "cancel_single_qubit_gates" optimization to the Pass Manager
pm = PassManager()
pm.append(QiskitVOQC(["cancel_single_qubit_gates", "replace_rzq"]))
new_circ = pm.run(circ)

# Print optimized circuit
print("\nAfter Optimization:")
print(new_circ)

Before Optimization:
     ┌───┐          ┌───┐       
q_0: ┤ T ├───────■──┤ T ├───────
     ├───┤┌───┐┌─┴─┐├───┤┌─────┐
q_1: ┤ T ├┤ H ├┤ X ├┤ H ├┤ TDG ├
     └───┘└───┘└───┘└───┘└─────┘

After Optimization:
               ┌───┐
q_0: ───────■──┤ S ├
     ┌───┐┌─┴─┐├───┤
q_1: ┤ H ├┤ X ├┤ H ├
     └───┘└───┘└───┘


### Two-qubit Gate Cancellation

*Two-qubit gate cancellation* is similar to single-qubit gate cancellation, except that it aims to commute and cancel CNOT gates. In the circuit below, the first CNOT gate commutes through the second, to cancel with the third.

In [10]:
# Build circuit with 3 qubits and 3 qates
circ = QuantumCircuit(3)
circ.cx(0, 1)
circ.cx(0, 2)
circ.cx(0, 1)
print("Before Optimization:")
print(circ)

# Append "cancel_two_qubit_gates" optimization to the Pass Manager
pm = PassManager()
pm.append(QiskitVOQC(["cancel_two_qubit_gates", "replace_rzq"]))
new_circ = pm.run(circ)

# Print optimized circuit
print("\nAfter Optimization:")
print(new_circ)

Before Optimization:
                    
q_0: ──■────■────■──
     ┌─┴─┐  │  ┌─┴─┐
q_1: ┤ X ├──┼──┤ X ├
     └───┘┌─┴─┐└───┘
q_2: ─────┤ X ├─────
          └───┘     

After Optimization:
          
q_0: ──■──
       │  
q_1: ──┼──
     ┌─┴─┐
q_2: ┤ X ├
     └───┘


### Rotation Merging

*Rotation merging* combines Rz gates that act on the same logical state (see discussion in Sec 4.4 of [our paper](https://arxiv.org/abs/1912.02250)). In the example below, the two Rz(pi/6) gates can be combines into a single Rz(pi/6 + pi/6) = Rz(pi/3) gate.

In [11]:
# Build circuit with 2 qubits and 4 gates
circ = QuantumCircuit(2)
circ.rz(pi/6, 1)
circ.cx(0, 1)
circ.cx(1, 0)
circ.rz(pi/6, 0)
print("Before Optimization:")
print(circ)

# Append "merge_rotations" optimization to the Pass Manager
pm = PassManager()
pm.append(QiskitVOQC(["merge_rotations", "replace_rzq"]))
new_circ = pm.run(circ)

# Print optimized circuit
print("\nAfter Optimization:")
print(new_circ)

Before Optimization:
                      ┌───┐┌──────────┐
q_0: ──────────────■──┤ X ├┤ RZ(pi/6) ├
     ┌──────────┐┌─┴─┐└─┬─┘└──────────┘
q_1: ┤ RZ(pi/6) ├┤ X ├──■──────────────
     └──────────┘└───┘                 

After Optimization:
                        ┌───┐
q_0: ────────────────■──┤ X ├
     ┌────────────┐┌─┴─┐└─┬─┘
q_1: ┤ RZ(1.0472) ├┤ X ├──■──
     └────────────┘└───┘     


### Hadamard Reduction

*Hadamard reduction* applies a series of identities to reduce the number of H gates in a circuit, for the purpose of making other optimizations (e.g. rotation merging) more effective. In the example below, we replace the first circuit with the second to remove two H gates.

In [12]:
# Build circuit with 2 qubits and 5 gates
circ = QuantumCircuit(2)
circ.h(1)
circ.sdg(1)
circ.cx(0, 1)
circ.s(1)
circ.h(1)

print("Before Optimization:")
print(circ)

# Append "hadamard_reduction" optimization to the Pass Manager
pm = PassManager()
pm.append(QiskitVOQC(["hadamard_reduction", "replace_rzq"]))
new_circ = pm.run(circ)

# Print optimized circuit
print("\nAfter Optimization:")
print(new_circ)

Before Optimization:
                                
q_0: ──────────────■────────────
     ┌───┐┌─────┐┌─┴─┐┌───┐┌───┐
q_1: ┤ H ├┤ SDG ├┤ X ├┤ S ├┤ H ├
     └───┘└─────┘└───┘└───┘└───┘

After Optimization:
                      
q_0: ───────■─────────
     ┌───┐┌─┴─┐┌─────┐
q_1: ┤ S ├┤ X ├┤ SDG ├
     └───┘└───┘└─────┘
